<a href="https://colab.research.google.com/github/Ranjani94/Machine_Learning/blob/master/Alternus_Vera/FinalNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Alternum Vera: Fake News Detection

- Course code : CMPE-257
- Team name : The Expendables


###Datasets

- Liar Liar Pants on Fire Dataset 
- Fake News Dataset
- News Category Dataset

###Github URL:

Political Bias: https://github.com/Vigneshthanga/MachineLearning-CMPE257/blob/master/AlternusVera_Sprint4_Political_Bias.ipynb 

Title Vs Body: https://github.com/Poohi5/AlternusVera 

Clickbait: https://github.com/Ranjani94/Machine_Learning/blob/master/Alternus_Vera/AlternusVera_Sprint4_Clickbait.ipynb 

NodeRank: https://github.com/akshaya-nagarajan/MachineLearning/blob/master/AlternusVeraSprint4NodeRank_Akshaya.ipynb 

###Process Steps:

 - Load the Data
 - Distillation Process
        - Data Preparation
        - Data Observation
        - Data Cleaning and Text Preprocessing
        - Data Visualization
        - Feature Extraction
        - TF-IDF Vectorization
        - LDA Topic Modelling
        - Classification
        - Extract Feature Vector
 - Classification Models
        - Random Forest Classifier
        - XGBoost classifier
        - Decision Tree Classifier
 - Ranking and Importance
 - Merge all features and individual contributions
 - Form Polynomial Equation

###Feature Selection:

1. Political Bias
2. Title Vs Body
3. Clickbait
4. Node Rank

###Individual Contributions

- Political Bias - Vignesh Kumar Thangarajan
- Title Vs Body - Pooja Patil
- Clickbait - Sivaranjani Kumar
- Node Rank - Akshaya Nagarajan

###Libraries Used

- NLTK
- Gensim
- Numpy
- Pandas
- CSV
- WordCloud
- Seaborn
- Scipy
- Regualr Expression
- Matplotlib
- Sklearn
- Keras

As a team, we decided on the importance of the factors presented above. We brainstormed on the general pre-processing techniques we did want to use. We also had common visualization methods and similar techniques for evaluating the classification model accuracy. Each of us enriched the dataset with individual features and persisted it in a csv file. Each feature vector is persisted on csv (which is distilled with LDA, sentiment scores). We also came up with a polynomial equation based on the factors and the accuracy scores we received by classification. The polynomial equation is then used to build a model for fake news classification. The polynomial equation that we have used is (0.4 * NodeRank) + (0.8 * Political Bias) + (0.7 * ClickBait) + (0.25 * TitlevsBody). The final model that we built is a variation of the stack ensemble technique. The trained models are saved in Pickle File and can be later used to predict new unseen data. We followed two approach to calculate the final class label for the data:



1.  With polynomial equation: As we discussed above, we calculated weights using accuracy using this formula : int(round(a1+a2+a3+a4)). The accuracy we got is 25.7 percent
2.  Without polynomial equation : We gone with the class that has majority votes. We made use of mode function in pandas dataframe. The accuracy is 36 percent.



## Mounting Google drive

In [0]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/MLSpring2020/"
base_dir = root_dir + 'the_expendables_AirQuality_Traffic'

Mounted at /content/gdrive


In [0]:
df = pd.read_csv(base_dir+'/Alternus_vera/Classification_4.csv')

In [0]:
df.head(4)

,Unnamed: 0,NodeRankClass,politicalbiasClass,clickbaitclass,ActualClass,titlebodyclass
0,0,FALSE,FALSE,FALSE,FALSE,half-true
1,1,half-true,mostly-true,half-true,half-true,half-true
2,2,mostly-true,mostly-true,mostly-true,mostly-true,half-true
3,3,mostly-true,half-true,FALSE,FALSE,half-true


In [0]:
df.shape

(3000, 6)

In [0]:
acc = [0.4, 0.8, 0.7, 0.2]

In [0]:
weight = [float(i)/sum(acc) for i in acc]

In [0]:
for i in range(len(weight)):
  weight[i] = round(weight[i], 2) 

In [0]:
weight

[0.19, 0.38, 0.33, 0.1]

In [0]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df['NodeRankClass'])

LabelEncoder()

In [0]:
df['NodeRank_label'] = label_encoder.transform(df['NodeRankClass'])
df['politicalbias_label'] = label_encoder.transform(df['politicalbiasClass'])
df['clickbait_label'] = label_encoder.transform(df['clickbaitclass'])
df['titlebody_label'] = label_encoder.transform(df['titlebodyclass'])

In [0]:
try:
  df.drop(columns=['Unnamed: 0'], inplace=True)
except e as Exception:
  pass

In [0]:
df.head(4)

,Unnamed: 0,NodeRankClass,politicalbiasClass,clickbaitclass,ActualClass,titlebodyclass,NodeRank_label,politicalbias_label,clickbait_label,titlebody_label
0,0,FALSE,FALSE,FALSE,FALSE,half-true,0,0,0,3
1,1,half-true,mostly-true,half-true,half-true,half-true,3,4,3,3
2,2,mostly-true,mostly-true,mostly-true,mostly-true,half-true,4,4,4,3
3,3,mostly-true,half-true,FALSE,FALSE,half-true,4,3,0,3


In [0]:
df_1 = df[['NodeRank_label', 'politicalbias_label','clickbait_label','titlebody_label']]

In [0]:
def calcLabel(x):
  a1 = x.NodeRank_label * weight[0]
  a2 = x.politicalbias_label * weight[1]
  a3 = x.clickbait_label * weight[2]
  a4 = x.titlebody_label * weight[3]
  s = int(round(a1+a2+a3+a4))
  return s

In [0]:
df_1.head(4)

,NodeRank_label,politicalbias_label,clickbait_label,titlebody_label
0,0,0,0,3
1,3,4,3,3
2,4,4,4,3
3,4,3,0,3


In [0]:
df_1['final_label'] = df_1.apply(lambda x: calcLabel(x), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df['label_class'] = label_encoder.inverse_transform(df_1['final_label'].values)

In [0]:
df.head(4)

,Unnamed: 0,NodeRankClass,politicalbiasClass,clickbaitclass,ActualClass,titlebodyclass,NodeRank_label,politicalbias_label,clickbait_label,titlebody_label,label_class
0,0,FALSE,FALSE,FALSE,FALSE,half-true,0,0,0,3,FALSE
1,1,half-true,mostly-true,half-true,half-true,half-true,3,4,3,3,half-true
2,2,mostly-true,mostly-true,mostly-true,mostly-true,half-true,4,4,4,3,mostly-true
3,3,mostly-true,half-true,FALSE,FALSE,half-true,4,3,0,3,barely-true


In [0]:
from sklearn.metrics import accuracy_score

## Accuracy score with Polynomial Equation

In [0]:
print('Accuracy Score is {}'.format(accuracy_score(df['ActualClass'], df['label_class'])*100))

Accuracy Score is 25.7


## Accuracy score without Polynomial Equation

In [0]:
df_2 = df[['NodeRankClass', 'politicalbiasClass','clickbaitclass','titlebodyclass']]

In [0]:
df_2['final_class'] = df_2.mode(axis=1)[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
print('Accuracy Score is {}'. format(round(accuracy_score(df['ActualClass'], df_2['final_class']), 2) * 100))

Accuracy Score is 36.0
